In [ ]:
# | default_exp blocks/mbconv_3d

# Imports

In [ ]:
# | export


import torch
from einops import rearrange
from torch import nn

from vision_architectures.blocks.cnn import CNNBlock3D, CNNBlock3DConfig
from vision_architectures.blocks.se import SEBlock3D
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import model_validator
from vision_architectures.utils.residuals import Residual

# Config

In [ ]:
# | export


class MBConv3DConfig(CNNBlock3DConfig):
    dim: int
    out_dim: int | None = None
    expansion_ratio: float = 6.0
    se_reduction_ratio: float = 4.0

    kernel_size: int = 3
    activation: str = "relu"
    normalization: str = "batchnorm3d"

    in_channels: None = None  # use dim instead
    out_channels: None = None  # use expansion_ratio instead

    @property
    def hidden_dim(self):
        return int(self.expansion_ratio * self.dim)

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        min_expansion_ratio = (self.dim + 1) / self.dim
        assert self.expansion_ratio > min_expansion_ratio, f"expansion_ratio must be greater than {min_expansion_ratio}"
        if self.out_dim is None:
            self.out_dim = self.dim
        return self

In [ ]:
MBConv3DConfig(dim=10, expansion_ratio=1.2).hidden_dim

12

# Architecture

In [ ]:
# | export


class MBConv3D(nn.Module):
    def __init__(self, config: MBConv3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = MBConv3DConfig.model_validate(config | kwargs)

        dim = self.config.dim
        hidden_dim = self.config.hidden_dim
        out_dim = self.config.out_dim
        se_reduction_ratio = self.config.se_reduction_ratio

        self.expand = CNNBlock3D(
            self.config,
            checkpointing_level,
            in_channels=dim,
            out_channels=hidden_dim,
            kernel_size=1,
            stride=1,
            padding=0,
        )
        self.depthwise_conv = CNNBlock3D(
            self.config,
            checkpointing_level,
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            conv_kwargs=self.config.conv_kwargs | dict(groups=hidden_dim),
        )
        self.se = SEBlock3D(dim=hidden_dim, r=se_reduction_ratio)
        self.pointwise_conv = CNNBlock3D(
            self.config,
            checkpointing_level,
            in_channels=hidden_dim,
            out_channels=out_dim,
            kernel_size=1,
            stride=1,
            padding=0,
            activation=None,
        )

        self.residual = Residual()

        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [dim], z, y, x, [dim])

        if not channels_first:
            x = rearrange(x, "b z y x d -> b d z y x").contiguous()

        # Now x is (b, dim, z, y, x)

        res_connection = x

        # Expand
        x = self.expand(x, channels_first=True)
        # (b, hidden_dim, z, y, x)

        # Depthwise Conv
        x = self.depthwise_conv(x, channels_first=True)
        # (b, hidden_dim, z, y, x)

        # SE
        x = self.se(x, channels_first=True)
        # (b, hidden_dim, z, y, x)

        # Pointwise Conv
        x = self.pointwise_conv(x, channels_first=True)
        # (b, dim, z, y, x)

        # Residual
        if x.shape == res_connection.shape:
            x = self.residual(x, res_connection)

        if not channels_first:
            x = rearrange(x, "b d z y x -> b z y x d").contiguous()
            # (b, z, y, x, dim)

        return x

    def forward(self, x: torch.Tensor, channels_first: bool = True):
        return self.checkpointing_level2(self._forward, x, channels_first)

In [ ]:
test = MBConv3D(dim=10)
display(test)

sample_input = torch.randn(2, 10, 32, 32, 32)
test(sample_input).shape


MBConv3D(
  (expand): CNNBlock3D(
    (cnn): Conv3d(10, 60, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    (norm_layer): BatchNorm3d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_layer): ReLU()
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (depthwise_conv): CNNBlock3D(
    (cnn): Conv3d(60, 60, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same, groups=60, bias=False)
    (norm_layer): BatchNorm3d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_layer): ReLU()
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (se): SEBlock3D(
    (squeeze): AdaptiveAvgPool3d(output_size=(1, 1, 1))
    (excitation): Sequential(
      (0): Conv3d(60, 15, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (1): BatchNorm3d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): Conv3d(15, 60, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (4): BatchNorm3d(60, eps=1e-05

torch.Size([2, 10, 32, 32, 32])

# nbdev

In [ ]:
!nbdev_export